In [1]:
import numpy as np
import pandas as pd
#!pip install folium
#!pip install geopy
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

**CLEANING DELHI**

In [2]:
df_delhi=pd.read_csv('delhi_dataSet.csv')
df_delrest=pd.read_csv('restaurant_dataSet.csv')
df_delhi.columns=['this','Borough','Neighborhood','latitude','longitude']
df_delhi.drop(['this'],axis=1,inplace=True)
df_delhi.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Begum Pur,NaN,NaN


In [3]:
df_delrest.columns=['this','Neighborhood','Neighborhood lat','Neighborhood lon','Venue','Venue lat','Venue lon','Venue category']
df_delrest.drop(['this'],axis=1,inplace=True)
print(df_delhi.shape,df_delrest.shape)
df_delrest.head()
df_del=pd.merge(df_delrest,df_delhi,on='Neighborhood')
df_del.drop(['latitude','longitude'],axis=1,inplace=True)

(185, 4) (849, 7)


In [4]:
df_del.head()

,Neighborhood,Neighborhood lat,Neighborhood lon,Venue,Venue lat,Venue lon,Venue category,Borough
0,Adarsh Nagar,28.614193,77.071541,Eagle Boys Pizza,28.615595,77.070784,Pizza Place,North West Delhi
1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant,North West Delhi
2,Adarsh Nagar,28.614193,77.071541,Bikano East Patel Nagar,28.616190,77.066978,Fast Food Restaurant,North West Delhi
3,Adarsh Nagar,28.614193,77.071541,McDonald's,28.616330,77.067034,Fast Food Restaurant,North West Delhi
4,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant,North West Delhi


In [5]:
delhi_venues=df_del[['Neighborhood','Neighborhood lat','Neighborhood lon','Venue','Venue lat','Venue lon','Venue category']]
delhi_venues.head()

,Neighborhood,Neighborhood lat,Neighborhood lon,Venue,Venue lat,Venue lon,Venue category
0,Adarsh Nagar,28.614193,77.071541,Eagle Boys Pizza,28.615595,77.070784,Pizza Place
1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
2,Adarsh Nagar,28.614193,77.071541,Bikano East Patel Nagar,28.616190,77.066978,Fast Food Restaurant
3,Adarsh Nagar,28.614193,77.071541,McDonald's,28.616330,77.067034,Fast Food Restaurant
4,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant


**CLEANING BANGALORE**

In [6]:
df_ban=pd.read_csv('bang.csv')
df_ban.columns=['this','Neighborhood','Latitude','Longitude']
df_ban.drop(['this'],axis=1,inplace=True)
df_ban.head()

,Neighborhood,Latitude,Longitude
0,Agram,45.813177,15.977048
1,Amruthahalli,13.066513,77.596624
2,Attur,11.663711,78.533551
3,Banaswadi,13.014162,77.651854
4,Bellandur,58.235358,26.683116


In [7]:
df_ban.shape

(201, 3)

**A MAP OF DELHI WITH NEIGHBORHOODS:**

In [8]:
address='Delhi, India'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

28.6273928 77.1716954


In [9]:
map_delhi=folium.Map(location=[latitude,longitude],zoom_start=12)
for lat,lon,label in zip(df_del['Neighborhood lat'],df_del['Neighborhood lon'],df_del['Neighborhood']):
    folium.CircleMarker(
    [lat,lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_delhi)
map_delhi    

**A MAP OF Bangalore WITH NEIGHBORHOODS:**

In [10]:
address='Bangalore, India'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

12.9791198 77.5912997


In [11]:
map_bangalore=folium.Map(location=[latitude,longitude],zoom_start=12)
for lat,lon,label in zip(df_ban['Latitude'],df_ban['Longitude'],df_ban['Neighborhood']):
    folium.CircleMarker(
    [lat,lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_bangalore)
map_bangalore   

In [12]:
#DEFINE FOURSQUARE CREDENTIALS:
CLIENT_ID = 'FU3OXQ2S3TAJBGRJKVMSDN31IHBMUWIT1TQORGLQVV4YQFNA' # your Foursquare ID
CLIENT_SECRET = 'DARJDUYHI1QPIC45U5T12V33TOHYM4MTJONIOAFMVXVVYEPM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FU3OXQ2S3TAJBGRJKVMSDN31IHBMUWIT1TQORGLQVV4YQFNA
CLIENT_SECRET:DARJDUYHI1QPIC45U5T12V33TOHYM4MTJONIOAFMVXVVYEPM


In [13]:
#GET THE NEIGHBORHOODS VENUES FOR BANGALORE:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
import requests
bangalore_venues = getNearbyVenues(names=df_ban['Neighborhood'],
                                   latitudes=df_ban['Latitude'],
                                   longitudes=df_ban['Longitude'])

Agram
Amruthahalli
Attur
Banaswadi
Bellandur
Bhattarahalli
Bidrahalli
Byatarayanapura
Devanagundi
Devasandra
Doddagubbi
Doddanekkundi
Domlur
EPIP
Fraser Town
Gunjur
HighCourt
Hoodi
Horamavu
Indiranagar S.O (Bangalore)
Jakkur
Kadugodi
Kalkunte
Kannamangala
Kodigehalli
Kothanur
Krishnarajapuram
Kundalahalli
Lingarajapuram
Mahadevapura
Medimallasandra
Mundur
Museum Road
NAL
Panathur
Rajanakunte
Sadashivanagar
Samethanahalli
Singanayakanahalli
Vasanthanagar
Venkateshapura
Vimanapura
Virgonagar
Whitefield
Yelahanka
Adugodi
Agara
Anjanapura
Banashankari
Bannerghatta
Basavanagudi H.O
Begur
Bolare
Bommanahalli S.O (Bangalore)
Chandapura
Chandapura
Chickpet
Chikkalasandra
Deepanjalinagar
Doddakallasandra
Girinagar S.O (Bangalore)
Gottigere
Haragadde
Hennagara
Hulimangala
Hulimavu
Hulimavu
Huskur
Jayanagar H.O
Jigani
Kalkere
Kallubalu
Kathriguppe
Kengeri
Konanakunte
Koramangala
Kumbalagodu
Madivala
Mallathahalli
Mavalli
Nayandahalli
Ragihalli
Ramohalli
Sakalavara
Shanthinagar
Singasandra
Subrama

In [15]:
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agram,45.813177,15.977048,Amélie,45.813842,15.979011,Dessert Shop
1,Agram,45.813177,15.977048,Trg bana Josipa Jelačića,45.813032,15.976868,Plaza
2,Agram,45.813177,15.977048,Tržnica Dolac,45.814070,15.977261,Farmers Market
3,Agram,45.813177,15.977048,Corner bar,45.812930,15.979440,Bar
4,Agram,45.813177,15.977048,Gajbica,45.813531,15.979550,Bistro


In [16]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Achitnagar,2,2,2,2,2,2
Adugodi,4,4,4,4,4,4
Agram,97,97,97,97,97,97
Akkur,20,20,20,20,20,20
Alahalli,1,1,1,1,1,1
...,...,...,...,...,...,...
Vimanapura,6,6,6,6,6,6
Virgonagar,2,2,2,2,2,2
Whitefield,6,6,6,6,6,6


In [17]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood lat,Neighborhood lon,Venue,Venue lat,Venue lon,Venue category
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,9,9,9,9,9,9
Anand Vihar,6,6,6,6,6,6
Ashok Vihar,4,4,4,4,4,4
Azadpur,4,4,4,4,4,4
...,...,...,...,...,...,...
Vasundhara Enclave,4,4,4,4,4,4
Vikaspuri,4,4,4,4,4,4
Vivek Vihar,1,1,1,1,1,1


**UNIQUE CATEGORIES OF VENUES IN BOTH THE PLACES**

In [18]:
print('There are {} uniques categories in Delhi.'.format(len(delhi_venues['Venue category'].unique())))
print('There are {} uniques categories in Bangalore.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 74 uniques categories in Delhi.
There are 159 uniques categories in Bangalore.


In [19]:
delhi_venues.head()

,Neighborhood,Neighborhood lat,Neighborhood lon,Venue,Venue lat,Venue lon,Venue category
0,Adarsh Nagar,28.614193,77.071541,Eagle Boys Pizza,28.615595,77.070784,Pizza Place
1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
2,Adarsh Nagar,28.614193,77.071541,Bikano East Patel Nagar,28.616190,77.066978,Fast Food Restaurant
3,Adarsh Nagar,28.614193,77.071541,McDonald's,28.616330,77.067034,Fast Food Restaurant
4,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant


**ANALYSING EACH NEIGHBORHOOD OF DELHI:**

In [20]:
#one hot encoding:
del_onehot=pd.get_dummies(delhi_venues[['Venue category']], prefix="", prefix_sep="")
del_onehot=pd.concat([delhi_venues['Neighborhood'],del_onehot],axis=1)
del_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Awadhi Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Soup Place,South Indian Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
del_onehot.shape

(913, 75)

In [22]:
del_grouped=del_onehot.groupby('Neighborhood').mean().reset_index()
del_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Awadhi Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Soup Place,South Indian Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Adarsh Nagar,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.00,0.0,0.0,0.0,0.222222,0.0,0.00,...,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Vihar,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.25,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Azadpur,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#TOP 5 MOST COMMON VENUES:
num_top_venues = 5

for hood in del_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = del_grouped[del_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Nagar----
                     venue  freq
0     Fast Food Restaurant  0.50
1        Indian Restaurant  0.25
2              Pizza Place  0.25
3       Mexican Restaurant  0.00
4  New American Restaurant  0.00


----Alaknanda----
                       venue  freq
0          Indian Restaurant  0.22
1                  BBQ Joint  0.22
2                Pizza Place  0.11
3  Middle Eastern Restaurant  0.11
4    New American Restaurant  0.11


----Anand Vihar----
                venue  freq
0   Indian Restaurant  0.33
1          Soup Place  0.17
2   Indian Sweet Shop  0.17
3         Pizza Place  0.17
4  Punjabi Restaurant  0.17


----Ashok Vihar----
                       venue  freq
0          Indian Restaurant  0.50
1                      Diner  0.25
2                     Bakery  0.25
3          Afghan Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Azadpur----
                    venue  freq
0  Argentinian Restaurant  0.25
1       Indian Restaurant  0.25
2              

**NOW LETS DISPLAY THE TOP 10 VENUES FOR EACH NEIGHBORHOOD:**

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = del_grouped['Neighborhood']

for ind in np.arange(del_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(del_grouped.iloc[ind, :], num_top_venues)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(116, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
1,Alaknanda,BBQ Joint,Indian Restaurant,New American Restaurant,Restaurant,Middle Eastern Restaurant,Pizza Place,Steakhouse,Deli / Bodega,Dhaba,Dim Sum Restaurant
2,Anand Vihar,Indian Restaurant,Pizza Place,Indian Sweet Shop,Soup Place,Punjabi Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Deli / Bodega,Dhaba,Dim Sum Restaurant
3,Ashok Vihar,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,Azadpur,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


**CLUSTER THE NEIGHBORHOODS TOGETHER:**

In [27]:
kclusters = 5
from sklearn.cluster import KMeans
del_clustering = del_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(del_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
del_clustering.head()

[3 0 0 1 2 0 0 0 0 3]


,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Awadhi Restaurant,BBQ Joint,Bagel Shop,Bakery,Bengali Restaurant,...,Soup Place,South Indian Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00,0.0,0.0,0.0,0.222222,0.0,0.00,0.0,...,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#adding cluster labels:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

del_merged = df_del

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
del_merged =del_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

del_merged.head()

,Neighborhood,Neighborhood lat,Neighborhood lon,Venue,Venue lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,28.614193,77.071541,Eagle Boys Pizza,28.615595,77.070784,Pizza Place,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
1,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
2,Adarsh Nagar,28.614193,77.071541,Bikano East Patel Nagar,28.616190,77.066978,Fast Food Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
3,Adarsh Nagar,28.614193,77.071541,McDonald's,28.616330,77.067034,Fast Food Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
4,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant,North West Delhi,1,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [32]:
address='Delhi, India'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

28.6273928 77.1716954


**FINALLY LET'S VISUALIZE THE CLUSTERS:**

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(del_merged['Venue lat'], del_merged['Venue lon'], del_merged['Neighborhood'], del_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**EXAMINE CLUSTERS:**

In [35]:
#CLUSTER 1:
del_merged.loc[del_merged['Cluster Labels'] == 0, del_merged.columns[[1] + list(range(5, del_merged.shape[1]))]]

,Neighborhood lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,28.720806,77.110130,Restaurant,North West Delhi,0,Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Eastern European Restaurant
50,28.659518,77.206671,Afghan Restaurant,North Delhi,0,Afghan Restaurant,Snack Place,Indian Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
51,28.659518,77.206671,Afghan Restaurant,Central Delhi,0,Afghan Restaurant,Snack Place,Indian Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
52,28.659518,77.207026,Afghan Restaurant,North Delhi,0,Afghan Restaurant,Snack Place,Indian Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
53,28.659518,77.207026,Afghan Restaurant,Central Delhi,0,Afghan Restaurant,Snack Place,Indian Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,28.631615,77.096328,Bakery,West Delhi,0,Indian Restaurant,Fried Chicken Joint,Food Court,Indian Sweet Shop,Fast Food Restaurant,Bakery,Pizza Place,Punjabi Restaurant,Donut Shop,Dhaba
875,28.642152,77.111674,BBQ Joint,West Delhi,0,Bakery,Food Truck,Indian Restaurant,BBQ Joint,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
876,28.642152,77.113799,Indian Restaurant,West Delhi,0,Bakery,Food Truck,Indian Restaurant,BBQ Joint,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
877,28.642152,77.113731,Food Truck,West Delhi,0,Bakery,Food Truck,Indian Restaurant,BBQ Joint,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [36]:
#CLUSTER 2:
del_merged.loc[del_merged['Cluster Labels'] == 1, del_merged.columns[[1] + list(range(5, del_merged.shape[1]))]]

,Neighborhood lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,28.699453,77.187832,Indian Restaurant,North West Delhi,1,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
5,28.699453,77.188716,Bakery,North West Delhi,1,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
6,28.699453,77.185953,Diner,North West Delhi,1,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
7,28.699453,77.189288,Indian Restaurant,North West Delhi,1,Indian Restaurant,Bakery,Diner,Falafel Restaurant,Dhaba,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
17,28.698041,77.139020,Indian Restaurant,North West Delhi,1,Indian Restaurant,Bakery,Food Court,Food,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,28.658844,77.088989,Indian Restaurant,West Delhi,1,Indian Restaurant,Bakery,Café,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
909,28.638419,77.075000,Pizza Place,West Delhi,1,Indian Restaurant,Pizza Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
910,28.638419,77.075190,Fast Food Restaurant,West Delhi,1,Indian Restaurant,Pizza Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
911,28.638419,77.074482,Indian Restaurant,West Delhi,1,Indian Restaurant,Pizza Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [38]:
#CLUSTER 3:
del_merged.loc[del_merged['Cluster Labels'] == 2, del_merged.columns[[1] + list(range(5, del_merged.shape[1]))]]

,Neighborhood lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,28.707657,77.172069,Argentinian Restaurant,North West Delhi,2,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
9,28.707657,77.171688,Indian Restaurant,North West Delhi,2,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
10,28.707657,77.179057,Café,North West Delhi,2,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
11,28.707657,77.172441,Restaurant,North West Delhi,2,Café,Argentinian Restaurant,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
12,28.688926,77.160298,Indian Restaurant,North West Delhi,2,Gastropub,Indian Restaurant,Café,Bakery,Food Truck,Food Stand,Food Court,Food,Fish & Chips Shop,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,28.651657,77.121416,Fast Food Restaurant,West Delhi,2,Fast Food Restaurant,Indian Restaurant,Café,Restaurant,Pizza Place,American Restaurant,Middle Eastern Restaurant,Snack Place,Diner,Vegetarian / Vegan Restaurant
900,28.651657,77.125525,Snack Place,West Delhi,2,Fast Food Restaurant,Indian Restaurant,Café,Restaurant,Pizza Place,American Restaurant,Middle Eastern Restaurant,Snack Place,Diner,Vegetarian / Vegan Restaurant
901,28.651657,77.119682,Indian Restaurant,West Delhi,2,Fast Food Restaurant,Indian Restaurant,Café,Restaurant,Pizza Place,American Restaurant,Middle Eastern Restaurant,Snack Place,Diner,Vegetarian / Vegan Restaurant
902,28.651657,77.119766,Café,West Delhi,2,Fast Food Restaurant,Indian Restaurant,Café,Restaurant,Pizza Place,American Restaurant,Middle Eastern Restaurant,Snack Place,Diner,Vegetarian / Vegan Restaurant


In [39]:
#CLUSTER 4:
del_merged.loc[del_merged['Cluster Labels'] == 3, del_merged.columns[[1] + list(range(5, del_merged.shape[1]))]]

,Neighborhood lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,28.614193,77.070784,Pizza Place,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
1,28.614193,77.076084,Indian Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
2,28.614193,77.066978,Fast Food Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
3,28.614193,77.067034,Fast Food Restaurant,North West Delhi,3,Fast Food Restaurant,Pizza Place,Indian Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
66,28.643483,77.223504,Fast Food Restaurant,North Delhi,3,Fast Food Restaurant,Chinese Restaurant,Food Court,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,28.639650,77.096502,Fast Food Restaurant,West Delhi,3,Fast Food Restaurant,Café,Restaurant,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
905,28.639650,77.097104,Fast Food Restaurant,West Delhi,3,Fast Food Restaurant,Café,Restaurant,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
906,28.639650,77.096946,Fast Food Restaurant,West Delhi,3,Fast Food Restaurant,Café,Restaurant,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
907,28.639650,77.097160,Restaurant,West Delhi,3,Fast Food Restaurant,Café,Restaurant,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [40]:
#CLUSTER 5:
del_merged.loc[del_merged['Cluster Labels'] == 4, del_merged.columns[[1] + list(range(5, del_merged.shape[1]))]]

,Neighborhood lat,Venue lon,Venue category,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
685,28.533479,77.292977,Café,South Delhi,4,Café,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
686,28.533479,77.291095,Café,South Delhi,4,Café,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
687,28.533479,77.290154,Café,South Delhi,4,Café,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
736,28.552146,77.228343,Café,South Delhi,4,Café,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
851,28.652368,77.151017,Café,West Delhi,4,Café,Sandwich Place,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Eastern European Restaurant
852,28.652368,77.148678,Sandwich Place,West Delhi,4,Café,Sandwich Place,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Eastern European Restaurant
853,28.652368,77.150221,Café,West Delhi,4,Café,Sandwich Place,Dumpling Restaurant,Deli / Bodega,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Eastern European Restaurant
866,28.659809,77.154631,Café,West Delhi,4,Café,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Dhaba,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant


**DOING THE SAME FOR BANGALORE:**

In [44]:
#ANALYSING EACH NEIGHBORHOOD:
#one hot encoding:
ban_onehot=pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")
ban_onehot=pd.concat([bangalore_venues['Neighborhood'],ban_onehot],axis=1)
ban_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Travel & Transport,Tunnel,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Agram,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agram,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agram,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agram,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agram,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
ban_grouped=ban_onehot.groupby('Neighborhood').mean().reset_index()
ban_grouped.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Travel & Transport,Tunnel,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Achitnagar,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,Adugodi,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.25,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Agram,0.0,0.0,0.0,0.010309,0.0,0.010309,0.0,0.010309,0.00,...,0.0,0.010309,0.0,0.0,0.0,0.0,0.020619,0.0,0.0,0.0
3,Akkur,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Alahalli,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [46]:
#TOP 5 MOST COMMON VENUES:
num_top_venues = 5

for hood in ban_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ban_grouped[ban_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Achitnagar----
           venue  freq
0   Burger Joint   0.5
1     Restaurant   0.5
2            ATM   0.0
3      Nightclub   0.0
4  Movie Theater   0.0


----Adugodi----
                venue  freq
0                 Gym  0.25
1         Bus Station  0.25
2                Café  0.25
3  Athletics & Sports  0.25
4                 ATM  0.00


----Agram----
                      venue  freq
0                       Bar  0.08
1                      Café  0.06
2                     Plaza  0.06
3                    Bistro  0.05
4  Mediterranean Restaurant  0.04


----Akkur----
              venue  freq
0       Supermarket   0.3
1  Department Store   0.1
2          Pharmacy   0.1
3         Pet Store   0.1
4    Discount Store   0.1


----Alahalli----
               venue  freq
0  Food & Drink Shop   1.0
1          Irish Pub   0.0
2    Motorcycle Shop   0.0
3           Mountain   0.0
4      Movie Theater   0.0


----Amruthahalli----
               venue  freq
0  Indian Restaurant  0.33
1   Dep

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ban_grouped['Neighborhood']

for ind in np.arange(ban_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ban_grouped.iloc[ind, :], num_top_venues)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(81, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Achitnagar,Burger Joint,Restaurant,Yoga Studio,Donut Shop,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
1,Adugodi,Bus Station,Café,Gym,Athletics & Sports,Yoga Studio,Dry Cleaner,Farmers Market,Farm,Escape Room,Eastern European Restaurant
2,Agram,Bar,Plaza,Café,Bistro,Historic Site,Dessert Shop,Mediterranean Restaurant,Pedestrian Plaza,Restaurant,Seafood Restaurant
3,Akkur,Supermarket,Pet Store,Bar,Pharmacy,Department Store,Chinese Restaurant,Discount Store,Sandwich Place,Escape Room,Eastern European Restaurant
4,Alahalli,Food & Drink Shop,Yoga Studio,Flower Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner


In [48]:
#CLUSTERING THE NEIGHBORHOODS TOGETHER:
kclusters = 5
#from sklearn.cluster import KMeans
ban_clustering = ban_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ban_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
ban_clustering.head()

[2 1 1 1 1 3 1 1 3 1]


,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Art Museum,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,...,Travel & Transport,Tunnel,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.010309,0.0,0.010309,0.0,0.010309,0.00,0.0,...,0.0,0.010309,0.0,0.0,0.0,0.0,0.020619,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [49]:
#adding cluster labels:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ban_merged = df_ban

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
ban_merged =ban_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ban_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,1.0,Bar,Plaza,Café,Bistro,Historic Site,Dessert Shop,Mediterranean Restaurant,Pedestrian Plaza,Restaurant,Seafood Restaurant
1,Amruthahalli,13.066513,77.596624,3.0,Indian Restaurant,Department Store,Shoe Store,Donut Shop,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
2,Attur,11.663711,78.533551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Banaswadi,13.014162,77.651854,3.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Kerala Restaurant,BBQ Joint,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Escape Room
4,Bellandur,58.235358,26.683116,2.0,Restaurant,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner


**VISUALIZE THE CLUSTERS:**

In [56]:
address='Bangalore, India'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

12.9791198 77.5912997


In [61]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ban_merged['Latitude'], ban_merged['Longitude'], ban_merged['Neighborhood'], ban_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

**EXAMINING THE CLUSTERS:**

In [63]:
#CLUSTER 1:

ban_merged.loc[ban_merged['Cluster Labels'] == 0, ban_merged.columns[[1] + list(range(5, ban_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,13.021626,Bakery,Yoga Studio,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant
31,10.835443,Yoga Studio,Flower Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
94,13.021626,Bakery,Yoga Studio,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant
165,13.230746,Bakery,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant


In [64]:
#CLUSTER 2:
ban_merged.loc[ban_merged['Cluster Labels'] == 1, ban_merged.columns[[1] + list(range(5, ban_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,45.813177,Plaza,Café,Bistro,Historic Site,Dessert Shop,Mediterranean Restaurant,Pedestrian Plaza,Restaurant,Seafood Restaurant
5,13.025800,Hotel,Japanese Restaurant,Café,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant
11,12.975720,Restaurant,Movie Theater,Fast Food Restaurant,Coffee Shop,Office,Sports Bar,Mediterranean Restaurant,Diner,Multiplex
12,12.962467,Café,Pizza Place,Chinese Restaurant,Food & Drink Shop,Sports Bar,Pub,BBQ Joint,Rajasthani Restaurant,Italian Restaurant
13,12.977673,Maharashtrian Restaurant,Cafeteria,Mediterranean Restaurant,Breakfast Spot,Ice Cream Shop,Hotel,Department Store,Dessert Shop,Financial or Legal Service
14,42.245363,Market,Flower Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
16,53.783508,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
19,12.973291,Lounge,Cupcake Shop,Restaurant,Café,Indian Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Brewery
20,13.078474,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner,Discount Store
28,13.011768,Bakery,Burger Joint,Andhra Restaurant,Garden Center,Café,Fast Food Restaurant,Farmers Market,Farm,Escape Room


In [65]:
#CLUSTER 3:
ban_merged.loc[ban_merged['Cluster Labels'] == 2, ban_merged.columns[[1] + list(range(5, ban_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,58.235358,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
84,58.235358,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
92,13.091176,Restaurant,Yoga Studio,Donut Shop,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner


In [66]:
#CLUSTER 4:
ban_merged.loc[ban_merged['Cluster Labels'] == 3, ban_merged.columns[[1] + list(range(5, ban_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,13.066513,Department Store,Shoe Store,Donut Shop,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
3,13.014162,Vegetarian / Vegan Restaurant,Bakery,Kerala Restaurant,BBQ Joint,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Escape Room
7,13.062074,Andhra Restaurant,Department Store,Flower Shop,Gourmet Shop,Kerala Restaurant,Cosmetics Shop,Wine Shop,History Museum,Donut Shop
17,12.991903,Bus Station,Yoga Studio,Hotel,Fast Food Restaurant,Farm,Escape Room,IT Services,Eastern European Restaurant,Dumpling Restaurant
18,13.027331,Chinese Restaurant,Athletics & Sports,Dry Cleaner,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room
27,12.968444,Bakery,Indian Restaurant,Hotel,Burger Joint,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant
38,13.147862,Café,Grocery Store,Cupcake Shop,Cosmetics Shop,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant
57,12.912192,Department Store,Bakery,Pizza Place,Gym,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant
61,12.856532,Department Store,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner
68,12.929273,Café,Juice Bar,Chinese Restaurant,Restaurant,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Candy Store,Shopping Mall


In [67]:
#CLUSTER 5:
ban_merged.loc[ban_merged['Cluster Labels'] == 4, ban_merged.columns[[1] + list(range(5, ban_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,12.821061,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner,Discount Store


COMPARING THE RESULTS:

WE CAN CLEARLY SEE THAT DELHI WINS IN THIS BATTLE AGAINST BANGALORE AS IT HAS A WIDE VARIETY OF RESTAURANTS TO CHOOSE FROM FOR A FOODIE.